In [ ]:
# Copyright 2024 Forusone
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Working with Parallel Function Calls and Multiple Function Responses in Gemini

This notebook simplifies the below original notebook written by Kristopher Overholt
[Working with Parallel Function Calls and Multiple Function Responses in Gemini](https://colab.sandbox.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/function-calling/parallel_function_calling.ipynb#scrollTo=VEqbX8OhE8y9)

## Overview

<img src="https://storage.googleapis.com/github-repo/generative-ai/gemini/function-calling/parallel-function-calling-in-gemini.png">

## Install Vertex AI SDK

In [1]:
%pip install --upgrade --user --quiet google-cloud-aiplatform wikipedia

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 33.5 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


## Set configuration

### Authenticate to access to the GCP

In [1]:

import sys
from IPython.display import Markdown, display

PROJECT_ID="ai-hangsik"
LOCATION="us-central1"

# For only colab user, no need this process for Colab Enterprise in Vertex AI.
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id=PROJECT_ID)

# set project.
!gcloud config set project {PROJECT_ID}

Updated property [core/project].


In [43]:
from typing import Any

from IPython.display import Markdown, display
from vertexai.generative_models import (
    FunctionDeclaration,
    GenerationConfig,
    GenerationResponse,
    GenerativeModel,
    Part,
    Tool,
)
import wikipedia

### Define helper function

In [44]:
# Helper function to extract one or more function calls from a Gemini Function Call response

def extract_function_calls(response: GenerationResponse) -> list[dict]:
    function_calls: list[dict] = []
    if response.candidates[0].function_calls:
        for function_call in response.candidates[0].function_calls:
            function_call_dict: dict[str, dict[str, Any]] = {function_call.name: {}}
            for key, value in function_call.args.items():
                function_call_dict[function_call.name][key] = value
            function_calls.append(function_call_dict)
    return function_calls

## Multiple function calls with a single functions

### Write function declarations and wrap them in a tool


In [45]:
search_wikipedia = FunctionDeclaration(
    name="search_wikipedia",
    description="Search for articles on Wikipedia",
    parameters={
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Query to search for on Wikipedia",
            },
        },
    },
)

wikipedia_tool = Tool(
    function_declarations=[
        search_wikipedia,
    ],
)

### Initialize the Gemini model

In [46]:
model = GenerativeModel(
    "gemini-1.5-pro",
    generation_config=GenerationConfig(temperature=0),
    tools=[wikipedia_tool],
)
chat = model.start_chat()

### Send prompt to Gemini

In [47]:
prompt = "Search for articles related to solar panels, renewable energy, and battery storage and provide a summary of your findings"
response = chat.send_message(prompt)
response

candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "search_wikipedia"
        args {
          fields {
            key: "query"
            value {
              string_value: "Solar panels"
            }
          }
        }
      }
    }
    parts {
      function_call {
        name: "search_wikipedia"
        args {
          fields {
            key: "query"
            value {
              string_value: "Renewable energy"
            }
          }
        }
      }
    }
    parts {
      function_call {
        name: "search_wikipedia"
        args {
          fields {
            key: "query"
            value {
              string_value: "Battery storage"
            }
          }
        }
      }
    }
  }
  avg_logprobs: -0.66181649102105033
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.0174560547
    severity: HARM_SEVERITY_NEGLIGIBLE
    seve

### Extract function names and parameters

In [48]:
function_calls = extract_function_calls(response)
function_calls

[{'search_wikipedia': {'query': 'Solar panels'}},
 {'search_wikipedia': {'query': 'Renewable energy'}},
 {'search_wikipedia': {'query': 'Battery storage'}}]

### Define function to search

In [49]:
def search_wikipedia(function_calls):
    # Make external API call
    result = wikipedia.summary(function_call["search_wikipedia"]["query"])

    return result

### Call the function three times

In [50]:
api_response = []

# Loop over multiple function calls
for function_call in function_calls:

    result = search_wikipedia(function_calls)

    # Make external API call
    # result = wikipedia.summary(function_call["search_wikipedia"]["query"])

    # Collect all API responses
    api_response.append(result)

api_response

['A solar panel is a device that converts sunlight into electricity by using photovoltaic (PV) cells. PV cells are made of materials that produce excited electrons when exposed to light. The electrons flow through a circuit and produce direct current (DC) electricity, which can be used to power various devices or be stored in batteries. Solar panels are also known as solar cell panels, solar electric panels, or PV modules.\nSolar panels are usually arranged in groups called arrays or systems. A photovoltaic system consists of one or more solar panels, an inverter that converts DC electricity to alternating current (AC) electricity, and sometimes other components such as controllers, meters, and trackers. Most panels are in solar farms or rooftop solar panels which  supply the electricity grid\nSome advantages of solar panels are that they use a renewable and clean source of energy, reduce greenhouse gas emissions, and lower electricity bills. Some disadvantages are that they depend on 

### Get a natural language summary

In [51]:
# Return the API response to Gemini
response = chat.send_message(
    [
        Part.from_function_response(
            name="search_wikipedia",
            response={
                "content": api_response[0],
            },
        ),
        Part.from_function_response(
            name="search_wikipedia",
            response={
                "content": api_response[1],
            },
        ),
        Part.from_function_response(
            name="search_wikipedia",
            response={
                "content": api_response[2],
            },
        ),
    ],
)
display(Markdown(response.text))

Solar panels are devices that convert sunlight into electricity using photovoltaic (PV) cells. They are arranged in arrays or systems and connected to inverters to convert DC electricity to AC electricity. Solar panels are a renewable and clean energy source that can reduce greenhouse gas emissions and lower electricity bills. However, they depend on sunlight availability, require cleaning, and have high initial costs.

Renewable energy, including solar, wind, and hydropower, is derived from natural resources that replenish over time. These energy sources are becoming increasingly efficient and cheaper, making them competitive with fossil fuels. Renewable energy sources help combat climate change by reducing greenhouse gas emissions and also minimize air pollution. However, challenges remain, such as fossil fuel subsidies, lobbying, and land use conflicts.

Battery storage systems store electrical energy using a group of batteries, providing a rapid response to grid fluctuations and stabilizing electricity supply. They are used for short-term peak power and ancillary services, minimizing power outages. Battery storage is cost-effective for up to two hours of use and is cheaper than open cycle gas turbine power. The levelized cost of storage has decreased significantly, making it an increasingly viable option for grid stability and reliability. 


## Parallel function calls across multiple functions

In [52]:
search_wikipedia = FunctionDeclaration(
    name="search_wikipedia",
    description="Search for articles on Wikipedia",
    parameters={
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Query to search for on Wikipedia",
            },
        },
    },
)

suggest_wikipedia = FunctionDeclaration(
    name="suggest_wikipedia",
    description="Get suggested titles from Wikipedia for a given term",
    parameters={
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Query to search for suggested titles on Wikipedia",
            },
        },
    },
)

summarize_wikipedia = FunctionDeclaration(
    name="summarize_wikipedia",
    description="Get article summaries from Wikipedia",
    parameters={
        "type": "object",
        "properties": {
            "topic": {
                "type": "string",
                "description": "Query to search for article summaries on Wikipedia",
            },
        },
    },
)

wikipedia_tool = Tool(
    function_declarations=[
        search_wikipedia,
        suggest_wikipedia,
        summarize_wikipedia,
    ],
)

### Initialize the Gemini model


In [53]:
model = GenerativeModel(
    "gemini-1.5-pro",
    generation_config=GenerationConfig(temperature=0),
    tools=[wikipedia_tool],
)
chat = model.start_chat()

### Send prompt to Gemini


In [54]:
prompt = "Show the search results, variations, and article summaries about Wikipedia articles related to the solar system"

response = chat.send_message(prompt)

### Extract function names and parameters

In [55]:
function_calls = extract_function_calls(response)
function_calls

[{'search_wikipedia': {'query': 'solar system'}},
 {'suggest_wikipedia': {'query': 'solar system'}},
 {'summarize_wikipedia': {'topic': 'solar system'}}]

### Make external API calls

In [56]:
api_response: dict[str, Any] = {}  # type: ignore

# Loop over multiple function calls
for function_call in function_calls:
    print(function_call)
    for function_name, function_args in function_call.items():
        # Determine which external API call to make
        if function_name == "search_wikipedia":
            result = wikipedia.search(function_args["query"])
        if function_name == "suggest_wikipedia":
            result = wikipedia.suggest(function_args["query"])
        if function_name == "summarize_wikipedia":
            result = wikipedia.summary(function_args["topic"], auto_suggest=False)

        # Collect all API responses
        api_response[function_name] = result

{'search_wikipedia': {'query': 'solar system'}}
{'suggest_wikipedia': {'query': 'solar system'}}
{'summarize_wikipedia': {'topic': 'solar system'}}


### Get a natural language summary

In [57]:
# Return the API response to Gemini
response = chat.send_message(
    [
        Part.from_function_response(
            name="search_wikipedia",
            response={
                "content": api_response.get("search_wikipedia", ""),
            },
        ),
        Part.from_function_response(
            name="suggest_wikipedia",
            response={
                "content": api_response.get("suggest_wikipedia", ""),
            },
        ),
        Part.from_function_response(
            name="summarize_wikipedia",
            response={
                "content": api_response.get("summarize_wikipedia", ""),
            },
        ),
    ],
)

display(Markdown(response.text))

Here's some information about the Solar System:

* **Search results:** Solar System, Formation and evolution of the Solar System, List of Solar System objects by size, Solar System (disambiguation), Photovoltaic system, List of Solar System objects, Exoplanet, Solar System belts, Passive solar building design, Small Solar System body
* **Variations:** soler system
* **Summary:** The Solar System, formed 4.6 billion years ago, is gravitationally bound by the Sun and objects orbiting it.  The Sun, a G-type main-sequence star, powers the system.  Eight planets, categorized as terrestrial (Mercury, Venus, Earth, Mars) or giant (gas giants Jupiter, Saturn; ice giants Uranus, Neptune), orbit the Sun.  The system also contains dwarf planets (Ceres, Pluto, Eris, etc.), asteroids, comets, and more, some within the asteroid belt (between Mars and Jupiter) or Kuiper belt (past Neptune).  The Sun's influence, the heliosphere, extends far, marked by the heliopause where it meets interstellar space.  The Oort cloud, a source of comets, lies even further out.  The Solar System resides within the Milky Way galaxy. 


And you're done! You successfully made parallel function calls for a couple of different use cases. Feel free to adapt the code samples here for your own use cases and applications. Or try another notebook to continue exploring other functionality in the Gemini API.

Happy parallel function calling!